In [ ]:
from enum import Enum
import requests

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import seaborn as sns

from serenity_sdk.client import load_local_config
from serenity_sdk.client import SerenityClient

%load_ext autoreload
%autoreload 2

pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_rows', 20)

class Environment(Enum):
    """
    The operational environment (e.g. test vs. production) to use for connection purposes.
    """
    DEV = 'dev'
    TEST = 'test'
    PRODUCTION = 'prod'

config_id = 'athansor'
config = load_local_config(config_id)

client = SerenityClient(config, env=Environment.DEV)

Let's start with a simple portfolio and some unit allocations.

In [ ]:
portfolio = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': 10000,
    'DOT': 50000
}

Next, fetch the asset master and map our portfolio the the Serenity internal IDs.

In [ ]:
refdata_response = client.call_api('refdata', '/asset/summaries')
df = pd.json_normalize(refdata_response['assetSummary']).set_index('assetId')
asset_master_df = df.filter(['assetId', 'nativeSymbol'])
ids_dict = {asset_master_df[asset_master_df['nativeSymbol'] == s].index[0]: s for s in portfolio.keys()}
portfolio_ids = {asset_master_df[asset_master_df.nativeSymbol == id].index[0]:qty for (id, qty) in portfolio.items()}
portfolio_ids

We'll use these Serenity asset IDs to build the query for the Serenity client.

In [ ]:
portfolio_input = {}
portfolio_input['assetPositions'] = [{'assetId': item[0], 'quantity': item[1]} for item in portfolio_ids.items()]

Note that in this exercise we do a point in time analysis, which is how the [Serenity API](https://developer.athansor.cloudwall.network/api/serenity-risk-services) is supposed to be used. If you need a timeseries of these data, you will need to loop through a set of dates.

In [ ]:
params = {'as_of_date': '2022-06-01'}

risk_attribution_json = client.call_api('risk', '/compute/attribution', body_json=portfolio_input, params=params)

The API respose will be a JSON file, with the information of the request, reference date, and the risk attribution

In [ ]:
risk_attribution_json.keys()

Initially, want to see how much of the portfolio volatility explained by the risk factors, and what is the specific (idiosyncratic) risk. This latter is the part of the portfolio volatility specific to the assets in the portfolio, and they cannot be explained by the used risk factors. We usually want to see most of the risk explained by the risk factors, which means the sources of risk (and thus returns) in our portfolio are known. We can do this decomposition for both volatility and for the variance. The advantage of using the variance is that it's additive, while volatility is easier to interpret (the measurement unit is %, instead of squared %).

In [ ]:
total_risk = pd.DataFrame.from_dict(risk_attribution_json['totalRisk'])
total_risk_table = total_risk.style.format("{:.1%}").bar()
total_risk_table

 The next step is to get marginal risk, which represents the percentage contribution of each asset’s volatility to the portfolio volatility. We can use this to understand what is the marginal contribution to the risk of each asset. *Marginal* means that it’s the impact on the portfolio volatility of a % increase of a certain asset’s weight.

In [ ]:
marginal_risk = pd.DataFrame.from_dict(
    risk_attribution_json['assetMarginalRisk'])
marginal_risk.set_index('assetId', inplace=True)
marginal_risk.set_index(marginal_risk.index.map(ids_dict), inplace=True)
marginal_risk_table = marginal_risk.style.format("{:.1%}").bar()
marginal_risk_table

We can weigh the marginal contribution by the asset weights to obtain the absolute contribution to the portfolio volatility of each asset. We can use this to understand how each asset is contributing to the volatility of the portfolio and make portfolio allocation decisions based on this information. 

In [ ]:
contribution_risk_absolute_byasset = pd.DataFrame.from_dict(
    risk_attribution_json['absoluteContributionRisk']['byAsset'])
contribution_risk_absolute_byasset.set_index('assetId', inplace=True)
contribution_risk_absolute_byasset.set_index(contribution_risk_absolute_byasset.index.map(ids_dict), inplace=True)
contribution_risk_absolute_byasset_table = contribution_risk_absolute_byasset.style.format("{:.1%}").bar()
contribution_risk_absolute_byasset_table

In [ ]:
contribution_risk_absolute_byasset = pd.DataFrame.from_dict(
    risk_attribution_json['absoluteContributionRisk']['byAsset'])
contribution_risk_absolute_byasset.set_index('assetId', inplace=True)
contribution_risk_absolute_byasset.set_index(contribution_risk_absolute_byasset.index.map(ids_dict), inplace=True)
contribution_risk_absolute_byasset_table = contribution_risk_absolute_byasset.style.format("{:.1%}").bar()
contribution_risk_absolute_byasset_table

Note how the absolute risk contribution sums up to the total volatility of the portfolio.

In [ ]:
contribution_risk_absolute_byasset.sum(axis=0)

In [ ]:
total_risk['volatility']

Given the derivation of this metric, we can use it to back-off our portfolio weights.

In [ ]:
portfolio_weights = contribution_risk_absolute_byasset[['factorRisk']] / marginal_risk[['factorRisk']]
portfolio_weights.rename(columns={"factorRisk": 'weights'}, inplace=True)
portfolio_weights_table = portfolio_weights.style.format("{:.1%}").bar()
portfolio_weights_table

In [ ]:
portfolio_weights = contribution_risk_absolute_byasset[['factorRisk']] / marginal_risk[['factorRisk']]
portfolio_weights.rename(columns={"factorRisk": 'weights'}, inplace=True)
portfolio_weights_table = portfolio_weights.style.format("{:.1%}").bar()
portfolio_weights_table

Likewise, we can decompose the portfolio *variance* to obtain the relative contribution to the portfolio risk. This helps to understand how much, in percentage terms, each asset is contributing to Factor, Specific and Total risk of the portfolio in terms of variance. If compared to the absolute contribution, here we use variance because it’s additive, while volatility is not additive. This can help to see if a certain asset has e.g. more Specific risk, which we might want to diversify or get exposure to. *Relative* here means that we are making a comparison to the other assets in the portfolio.

In [ ]:
contribution_risk_relative_byasset = pd.DataFrame.from_dict(
    risk_attribution_json['relativeContributionRisk']['byAsset'])
contribution_risk_relative_byasset.set_index('assetId', inplace=True)
contribution_risk_relative_byasset.set_index(contribution_risk_relative_byasset.index.map(ids_dict), inplace=True)
contribution_risk_relative_byasset_table = contribution_risk_relative_byasset.style.format("{:.1%}").bar()
contribution_risk_relative_byasset_table

And here we can also show that this metric sums up to the total portfolio risk (in terms of variance this time).

In [ ]:
contribution_risk_relative_byasset.sum(axis=0)

In [ ]:
factor_risk_test = total_risk.loc['factorRisk']['variance'] / total_risk.loc['totalRisk']['variance']
specific_risk_test = total_risk.loc['specificRisk']['variance'] / total_risk.loc['totalRisk']['variance']
print(factor_risk_test.round(10) == contribution_risk_relative_byasset.sum(axis=0)['factorRisk'].round(10))
print(specific_risk_test.round(10) == contribution_risk_relative_byasset.sum(axis=0)['specificRisk'].round(10))

Here's a way to get these metrics by sector. In this model we used the [DACS sector taxonomy](https://www.coindesk.com/static/coindesk-dacs/), with three sub-levels. The `parentSector` column tells us if the specific row has a parent sector. You can extract the full taxonomy from the `/sector/taxonomies` endpoint ([docs](https://developer.athansor.cloudwall.network/api/serenity-middleware#operation/list_sector_taxonomies_sector_taxonomies_get)).

In [ ]:
contribution_risk_relative_bysector = pd.DataFrame.from_dict(
    risk_attribution_json['relativeContributionRisk']['bySector'])
contribution_risk_relative_bysector

Now we can also get the marginal, absolute, and relatiev contributions to the portfolio risk from each of the factors. The definitions of these metrics are the same as above. Here you can also get the factor exposure, which is the dot product between asset weights and each asset's factor loading. This, in practice, gives you the portfolio factor exposure in the selected point in time.

In [ ]:
factor_risk = pd.DataFrame.from_dict(risk_attribution_json['factorRisk'])
factor_risk = factor_risk.set_index('factor').transpose()
factor_risk_table = factor_risk.style.format("{:.1%}").bar()
factor_risk_table

Note that the relative contribution sums to 100%, while the absolute contribution sums to the portfolio volatility explained by the factor risk, while the absolute contribution

In [ ]:
factor_risk.loc['relativeContribution'].sum().round(10) == 1

In [ ]:
factor_risk.loc['absoluteContribution'].sum().round(10) == total_risk['volatility']['factorRisk'].round(10)

For the derivations, we get the relative risk contribution from the absolute contribution. And we get the absolute contribution from the marginal contribution and from the factor exposure. If you want to know how to derive the marginal contribution, follow along, as it's explained later.

In [ ]:
factor_risk.loc['absoluteContribution'] / total_risk['volatility']['factorRisk']

In [ ]:
factor_risk.loc['marginalContribution'] * factor_risk.loc['factorExposure']

We can also aggregate the portfolio factor exposures by sector. This is useful to see how each sector exposure in our portfolio is contributing to each portfolio factor exposure. You can see that the sum of each Factor, Specific, or Total risk across the sectors will give you the portfolio factor exposure for each factor (we'll leave this exercise to you).

In [ ]:
sector_factors = pd.DataFrame.from_dict(risk_attribution_json['sectorFactorExposure'])
df = sector_factors.explode('factorRisk').reset_index(drop=True)
df = df.join(pd.json_normalize(df.factorRisk)).drop(columns=['factorRisk'])
df

From the Risk API you can also get the intermediate results that go into the factor model. Here you can, for instance, see the asset covariance matrix. It's fetched for the full universe, and we then subset it to the example portfolio.

In [ ]:
assetcov_response = client.call_api('risk', '/asset/covariance', params=params)

In [ ]:
cov_df = pd.DataFrame.from_dict(assetcov_response['matrix']).dropna()
cov_df = cov_df[cov_df['assetId1'].isin(ids_dict.keys()) & cov_df['assetId2'].isin(ids_dict.keys())]
cov_df = cov_df.pivot(index='assetId1', columns='assetId2', values='value')
cov_df.set_index(cov_df.index.map(ids_dict), inplace=True)
cov_df.columns = cov_df.columns.map(ids_dict)
sns.heatmap(cov_df, cmap='YlGnBu')

You can also fetch the factor returns that go into our risk factor model, in case you want to see how each factor performs at the selected point in time.

In [ ]:
factor_returns_response = client.call_api('risk', '/factor/returns', params=params)

In [ ]:
factor_returns = pd.DataFrame.from_dict(factor_returns_response['factorReturns']).pivot(index='closeDate', columns='factor', values='value')
factor_returns.style.format("{:.1%}")

We can also get the factor exposures of each asset in your portfolio.

In [ ]:
factor_exposures_response = client.call_api('risk', '/asset/factor/exposures', params=params)

In [ ]:
factor_exposures = pd.DataFrame.from_dict(factor_exposures_response['matrix'])
factor_exposures = factor_exposures[factor_exposures['assetId'].isin(ids_dict.keys())]
factor_exposures = factor_exposures.pivot(index='assetId', columns='factor', values='value')
factor_exposures.set_index(factor_exposures.index.map(ids_dict), inplace=True)
factor_exposures

You can use this along with the portfolio weights to get the portfolio factor exposure for each factor. This is an example for the Liquidity factor.

In [ ]:
np.dot(portfolio_weights['weights'], factor_exposures['liquidity'])

In [ ]:
factor_risk.loc['factorExposure']['liquidity']

In [ ]:
factorcov_response = client.call_api('risk', '/factor/covariance', params=params)

Earlier we've mentioned that we can also derive our marginal risk factor contribution. For this, we need to fetch the factor covariance matrix, which is the covariance matrix derived from the factor returns.

In [ ]:
fac_cov_df = pd.DataFrame.from_dict(factorcov_response['matrix']).dropna()
fac_cov_df = fac_cov_df.pivot(index='factor1', columns='factor2', values='value')
sns.heatmap(fac_cov_df, cmap='YlGnBu')
fac_cov_df

In [ ]:
fac_cov_df.loc['liquidity'] @ factor_risk.loc['factorExposure'] / total_risk['volatility']['factorRisk']

In [ ]:
factor_risk.loc['marginalContribution']['liquidity']

Here's an example portfolio for an exercise. It was a different formatting, you can play around with the taxonomies mapping to format this as the first portfolio example. Note how MLN and YFI impact the portfolio risk!

In [ ]:
portfolio_input = {
    "assetPositions": [
        {
            "assetId": "78e2e8e2-419d-4515-9b6a-3d5ff1448e89",
            "quantity": 5.1234564
        },
        {
            "assetId": "a957470a-5168-4f57-bcc2-99803984eab8",
            "quantity": 2221.5434535
        },
        {
            "assetId": "95b89dfd-c5f6-41d6-83c6-a9d97e0ac361",
            "quantity": 110.2344
        },
        {
            "assetId": "c8212fb7-8097-4c9c-818a-991ee1bfd70b",
            "quantity": 1653.9865
        },
        {
            "assetId": "73e2dfab-36b6-4859-87e1-78660f05af4a",
            "quantity": 9865.572146
        },
        {
            "assetId": "baa3592a-6406-4b52-8cb1-a73ff723b73f",
            "quantity": 182.43234
        },
        {
            "assetId": "18cf1e31-810e-4615-bbf4-853a2ff8b3d3",
            "quantity": 123023.6455
        },
        {
            "assetId": "e8f606a7-4d78-404a-9ae8-0be3f3cf0185",
            "quantity": 5672.2313
        },
        {
            "assetId": "51f5271a-41f8-4683-a269-ab55fcce10d9",
            "quantity": 7865.572146
        },
        {
            "assetId": "34a1d617-e699-4e50-b4fe-38958b6bdb8e",
            "quantity": 21365.572146
        },
        {
            "assetId": "caca1f29-cb39-4fff-a959-2e7b6b46bcfe",
            "quantity": 93865.572146
        },
        {
            "assetId": "9831dc92-cd68-42e0-8099-44d03153a8c0",
            "quantity": 64579.3781
        },
        {
            "assetId": "659aa572-0fd2-4490-a075-d6ce5809eb8c",
            "quantity": 1111.6666
        },
        {
            "assetId": "df53b0a7-f67b-41c0-b117-eff13edbe365",
            "quantity": 119865.572146
        },
        {
            "assetId": "99ff8d08-4d99-4010-a751-82c546d033d3",
            "quantity": 239865.572146
        },
        {
            "assetId": "f598493e-f802-4d5a-990d-de24b7a551dd",
            "quantity": 865.572146
        },
        {
            "assetId": "93d2ebc2-836f-4338-81f8-290d855700e9",
            "quantity": 59865.572146
        },
        {
            "assetId": "f1363f04-e39b-41fb-9143-dd3699940d00",
            "quantity": 7865.572146
        },
        {
            "assetId": "c0809ae9-e4ec-4506-bf38-fb233b09b685",
            "quantity": 22865.572146
        },
        {
            "assetId": "7ff76a6e-2d9e-4e14-869c-e60db708b5d3",
            "quantity": 42358.572146
        }
    ]
}